<a href="https://colab.research.google.com/github/antonio98s/SQL-Python-Portfolio-for-Data-Security-Analyst/blob/main/python_sql_portfolio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
mlg_ulb_creditcardfraud_path = kagglehub.dataset_download('mlg-ulb/creditcardfraud')

print('Data source import complete.')


In [ ]:
!pip install --upgrade scikit-learn
!pip install --upgrade imbalanced-learn

In [ ]:
!wget -O creditcard.csv "https://raw.githubusercontent.com/nsethi31/Kaggle-Data-Credit-Card-Fraud-Detection/master/creditcard.csv"

In [ ]:
!ls -lh creditcard.csv


In [ ]:
import pandas as pd

# The exact path on Kaggle
df = pd.read_csv('/kaggle/input/creditcardfraud/creditcard.csv')

print(df.head())
print(df.shape)  # Should show (284807, 31)
print("Fraud cases:", df['Class'].sum())  # Should be 492

In [ ]:
import pandas as pd
import sqlite3

# Load dataset (replace with your file path)
df = pd.read_csv('creditcard.csv')  # From Kaggle dataset

# Create SQLite database
conn = sqlite3.connect('fraud.db')
df.to_sql('transactions', conn, if_exists='replace', index=False)

# Sample SQL Query: Find transactions with amounts > 3x average and flag as potential fraud
query = """
SELECT Time, Amount, Class
FROM transactions
WHERE Amount > (SELECT AVG(Amount) * 3 FROM transactions)
ORDER BY Amount DESC
LIMIT 10;
"""

results = pd.read_sql_query(query, conn)
print(results)  # Visualize or export to CSV

# Advanced Query: Group by time windows to detect velocity fraud
velocity_query = """
SELECT strftime('%H', Time) AS Hour, COUNT(*) AS Transaction_Count
FROM transactions
GROUP BY Hour
HAVING Transaction_Count > 100  -- Threshold for suspicion
ORDER BY Transaction_Count DESC;
"""

velocity_results = pd.read_sql_query(velocity_query, conn)
print(velocity_results)

conn.close()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Features and target
X = df.drop('Class', axis=1)
y = df['Class']

# Split the data (stratify to keep fraud ratio in test set)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Train Random Forest with class weights to handle imbalance
model = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42, n_jobs=-1)
model.fit(X_train, y_train)

# Predict and evaluate
predictions = model.predict(X_test)
print(classification_report(y_test, predictions))
print("Confusion Matrix:\n", confusion_matrix(y_test, predictions))

In [ ]:
import matplotlib.pyplot as plt

importances = model.feature_importances_
feature_names = X.columns
indices = importances.argsort()[::-1]

plt.figure(figsize=(12, 6))
plt.title("Top 10 Feature Importances in Fraud Detection")
plt.bar(range(10), importances[indices[:10]])
plt.xticks(range(10), [feature_names[i] for i in indices[:10]], rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import sqlite3
import plotly.express as px

conn = sqlite3.connect('fraud.db')
df = pd.read_sql_query("SELECT * FROM transactions WHERE Class = 1", conn)  # Fraud only

fig = px.histogram(df, x='Amount', title='Distribution of Fraudulent Transaction Amounts')
fig.show()

# For a full dashboard, use: pip install streamlit
# Then run: streamlit run app.py with code to display queries and charts.